In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
import wandb
wandb.login(key="279a68e0fd5d16d5893ca46bdc25076ad1f3be50")
wandb.init(project="DLAssignment3",entity="cs22m072")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
import os
import wandb
wandb.login(key="279a68e0fd5d16d5893ca46bdc25076ad1f3be50")
wandb.init(project="DLAssignment3",entity="cs22m072")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m072. Use `wandb login --relogin` to force relogin


In [ ]:
class MyDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file,names=["English","Hindi"],header=None)
        
    def __getitem__(self, index):
        x = self.data.iloc[index]["English"]
        y = self.data.iloc[index]["Hindi"]
        return x, y
    
    def __len__(self):
        return len(self.data)


In [ ]:
train_data = MyDataset(os.getcwd()+'/hin_train.csv')
test_data = MyDataset(os.getcwd()+'/hin_test.csv')
val_data = MyDataset(os.getcwd()+'/hin_valid.csv')

In [ ]:

ENGLEN=32
HINDILEN=32
BATCH_SIZE=128
englishwords=torch.full((len(train_data), ENGLEN), 2).to(device)
hindiwords=torch.full((len(train_data), HINDILEN), 2).to(device)


# Preprocessing

In [ ]:
hindivocab=set()
englishvocab=set()
for x,y in train_data:
    for letter in x:
        englishvocab.add(letter)
    for letter in y:
        hindivocab.add(letter)  
for x,y in test_data:
    for letter in x:
        englishvocab.add(letter)
    for letter in y:
        hindivocab.add(letter)
for x,y in test_data:
    for letter in x:
        englishvocab.add(letter)
    for letter in y:
        hindivocab.add(letter)
hindivocab=list(hindivocab)
hindivocab.sort()
englishvocab=list(englishvocab)
englishvocab.sort()
hindivocab.insert(0,'0')#start
hindivocab.insert(1,'1') #end
hindivocab.insert(2,'2') #pad
englishvocab.insert(0,'0')#start
englishvocab.insert(1,'1') #end
englishvocab.insert(2,'2') #pad
print(englishvocab)
hindidictc={}
englishdictc={}
hindidicti={}
englishdicti={}
for i in range(len(hindivocab)):
    hindidicti[i]=hindivocab[i]
    hindidictc[hindivocab[i]]=i
for i in range(len(englishvocab)):
    englishdicti[i]=englishvocab[i]
    englishdictc[englishvocab[i]]=i

c=0
for x,y in train_data:
    for i in range(len(x)):
        englishwords[c][i]=englishdictc[x[i]]
    for i in range(len(y)):
        hindiwords[c][i]=hindidictc[y[i]]
    hindiwords[c][i+1]=1
    c+=1

englishwordsval=torch.full((len(val_data), ENGLEN), 2).to(device)
hindiwordsval=torch.full((len(val_data), HINDILEN), 2).to(device)
c=0
for x,y in val_data:
    for i in range(len(x)):
        englishwordsval[c][i]=englishdictc[x[i]]
    for i in range(len(y)):
        hindiwordsval[c][i]=hindidictc[y[i]]
    hindiwordsval[c][i+1]=1
    c+=1
    
englishwordstest=torch.full((len(test_data), ENGLEN), 2).to(device)
hindiwordstest=torch.full((len(test_data), HINDILEN), 2).to(device)
c=0
for x,y in test_data:
    for i in range(len(x)):
        englishwordstest[c][i]=englishdictc[x[i]]
    for i in range(len(y)):
        hindiwordstest[c][i]=hindidictc[y[i]]
    hindiwordstest[c][i+1]=1
    c+=1

['0', '1', '2', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# Encoder Decoder Seq2seq

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size,hidden_size,embedding_size,num_layers,typecell,dropout,bidirectional):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout)
        self.num_layers=num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.bidirectional=bidirectional
        self.p=dropout
        self.typecell=typecell
        if self.typecell=="gru":
            self.step = nn.GRU(embedding_size, hidden_size,num_layers,dropout=self.p,bidirectional=self.bidirectional)    
        if self.typecell=="lstm":
            self.step = nn.LSTM(embedding_size, hidden_size,num_layers,dropout=self.p,bidirectional=self.bidirectional)
        if self.typecell=="rnn":
            self.step = nn.RNN(embedding_size, hidden_size,num_layers,dropout=self.p,bidirectional=self.bidirectional) 

    def forward(self, inp, hidden,cell=None):
        embedded = self.dropout(self.embedding(inp))
        if self.typecell=="gru":
            output, hidden = self.step(embedded, hidden)   
            return output,hidden
        if self.typecell=="rnn":
            output, hidden = self.step(embedded, hidden)   
            return output,hidden
        if self.typecell=="lstm": 
            output, (hidden,cell) = self.step(embedded, (hidden,cell))
            return output, (hidden,cell)

    def initHidden(self):
        #for bidirection
        num_layers=self.num_layers
        if self.bidirectional:
            num_layers=self.num_layers*2
        hidden=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
        if self.typecell=="lstm":
            cell=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
            return (hidden,cell)        
        return hidden


    
class DecoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_layers,output_size,typecell,dropout):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout)
        self.num_layers=num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.output_size=output_size
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=2)
        self.p=dropout
        self.typecell=typecell
        if self.typecell=="gru":
            self.step = nn.GRU(embedding_size, hidden_size,num_layers,dropout=self.p)   
        if self.typecell=="rnn":
            self.step = nn.RNN(embedding_size, hidden_size,num_layers,dropout=self.p) 
        if self.typecell=="lstm":
            self.step = nn.LSTM(embedding_size, hidden_size,num_layers,dropout=self.p)

    def forward(self, inp, hidden,cell=None):
        embedded = self.dropout(self.embedding(inp))
        if self.typecell=="gru":
            output, hidden = self.step(embedded, hidden)   
            output1=self.out(output)
            return output1,hidden
        if self.typecell=="rnn":
            output, hidden = self.step(embedded, hidden)   
            output1=self.out(output)
            return output1,hidden
        if self.typecell=="lstm": 
            output, (hidden,cell) = self.step(embedded,  (hidden,cell))
            output1=self.out(output)
            return output1, (hidden,cell)

    def initHidden(self):
        #for bidirection
        num_layers=self.num_layers
        hidden=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
        if self.typecell=="lstm":
            cell=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
            return (hidden,cell)        
        return hidden 
            
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder,hencoder,cell=None):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.hencoder=hencoder
        self.cell=cell
    def forward(self, inp, target,teacher_force_ratio):
        outputs = torch.zeros(HINDILEN,BATCH_SIZE ,len(hindivocab)).to(device)
        if self.encoder.typecell=="lstm":
            p,(hencoder3d,cell)=self.encoder.forward(inp.to(device),self.hencoder,self.cell)
        else:
            p,hencoder3d=self.encoder.forward(inp.to(device),self.hencoder)
        tempdecoder=torch.zeros(self.encoder.num_layers,BATCH_SIZE,hencoder3d.size()[2]).to(device)
        tempdecoder[0]=hencoder3d[hencoder3d.size()[0]//2-1]
        tempdecoder[1]=hencoder3d[(hencoder3d.size()[0]//2)*2-1]
        hdecoder=torch.add(tempdecoder[0],tempdecoder[1])
        hdecoder=hdecoder.repeat(self.decoder.num_layers,1,1)
        if self.encoder.typecell=="lstm":
            tempcell=torch.zeros(2,BATCH_SIZE,cell.size()[2]).to(device)
            tempcell[0]=hencoder3d[cell.size()[0]//2-1]
            tempcell[1]=hencoder3d[(cell.size()[0]//2)*2-1]
            cell=torch.add(tempcell[0],tempcell[1])
            cell=cell.repeat(self.decoder.num_layers,1,1)
        x=torch.full((1,BATCH_SIZE),hindidictc['0'])
        if self.encoder.typecell=="lstm":
            output,(hdecoder,cell)=self.decoder.forward(x.to(device),hdecoder,cell)
        else:
            output,hdecoder=self.decoder.forward(x.to(device),hdecoder)
        outputs[0]=output
        t=1
        for i in range(1,HINDILEN):
            if random.random() > teacher_force_ratio:
                output=self.decoder.softmax(output)
                nextinp=torch.argmax(output, dim=2)
                if self.encoder.typecell=="lstm":
                    output,(hdecoder,cell)=self.decoder.forward(nextinp.to(device),hdecoder,cell)
                else:
                    output,hdecoder=self.decoder.forward(nextinp.to(device),hdecoder)
                outputs[t]=output
                t+=1
            else:
                nextinp=target[i-1,:].unsqueeze(0)
                if self.encoder.typecell=="lstm":
                    output,(hdecoder,cell)=self.decoder.forward(nextinp.to(device),hdecoder,cell)
                else:
                    output,hdecoder=self.decoder.forward(nextinp.to(device),hdecoder)
                outputs[t]=output
                t+=1
        return outputs
        
    
alpha=[]
def train(encoder,decoder,seq2seq,epoch):
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(reduction="sum")
    loss=0
    count=0
    numbatches=englishwords.shape[0]//BATCH_SIZE
    for ep in range(epoch):
        trainloss=0
        train_correct=0
        for i in range(numbatches):
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            temp=englishwords[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
            temph=hindiwords[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
            temp=temp.t()
            temph=temph.t()
            output=seq2seq.forward(temp,temph,0.5)
            train_correct+=train_accuracy(output,temph)
            output = output[:].reshape(-1, output.shape[2])
            tem = temph[:].reshape(-1)
            loss=criterion(output,tem)
            loss.backward()
            trainloss+=loss.item()
            torch.nn.utils.clip_grad_norm_(decoder.parameters(),max_norm = 1)
            torch.nn.utils.clip_grad_norm_(encoder.parameters(),max_norm = 1)
            encoder_optimizer.step()
            decoder_optimizer.step()
        val_correct,cur_loss=accuracy(seq2seq,englishwordsval,hindiwordsval)
        print(ep,trainloss/(51200*HINDILEN),cur_loss/(4096*HINDILEN),val_correct,train_correct)
        trainloss=trainloss/(51200*HINDILEN)
        cur_loss=cur_loss/(4096*HINDILEN)
        tra_acc=train_correct/51200
        val_acc=val_correct/4096
        wandb.log({"Training loss":trainloss,'Val loss':cur_loss,'Training Accuracy':tra_acc,'Val Accuracy':val_acc})

def train_accuracy(output,temph):
        output=nn.Softmax(dim=2)(output)
        output=torch.argmax(output,dim=2)
        temph=temph.t()
        output=output.t()
        correct=0
        for i in range(BATCH_SIZE):
            if(torch.equal(output[i],temph[i])):
                correct+=1
        return correct
        
def accuracy(seq2seq,english,hindi):
    numbatches=english.shape[0]//BATCH_SIZE
    correct=0
    criterion = nn.CrossEntropyLoss(reduction="sum")
    loss=0
    for i in range(numbatches):
        temp=english[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        temph=hindi[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        temp=temp.t()
        temph=temph.t()
        output=seq2seq.forward(temp,temph,0)
        o = output[:].reshape(-1, output.shape[2])
        tem = temph[:].reshape(-1)
        x=criterion(o,tem)
        loss+=x.item()
        output=nn.Softmax(dim=2)(output)
        output=torch.argmax(output,dim=2)
        temph=temph.t()
        output=output.t()
        for i in range(BATCH_SIZE):
            if(torch.equal(output[i],temph[i])):
                correct+=1

    return correct,loss


# Sweep code

In [ ]:
def sweep():
    wandb.init(project='DLAssignment3')
    config = wandb.config
    wandb.run.name = "cellType_{}_embSize_{}_layers_{}_batchSize_{}_hidden_{}_dropout_{}".format(config.cell_type,config.input_embedding_size,config.no_of_layers,config.batchsize,config.hidden_size,config.dropout)
    hidden_size = config.hidden_size
    char_embed_size = config.input_embedding_size
    no_of_layers = config.no_of_layers
    epochs = config.epochs
    batchsize = config.batchsize
    dropout = config.dropout
    cell_type=config.cell_type
    if(config.bidirectional=='Yes'):
        bidirectional=True
    else:
        bidirectional=False
    encoder=EncoderRNN(len(englishvocab),hidden_size,char_embed_size,no_of_layers,cell_type,dropout,bidirectional).to(device)
    decoder=DecoderRNN(len(hindivocab),hidden_size,char_embed_size,no_of_layers,len(hindivocab),cell_type,dropout).to(device)
    if cell_type=='lstm':
        hencoder,cell=encoder.initHidden()
        seq2seq=Seq2Seq(encoder,decoder,hencoder,cell)
    else:
        hencoder=encoder.initHidden()
        seq2seq=Seq2Seq(encoder,decoder,hencoder)
    train(encoder,decoder,seq2seq,epochs)

In [ ]:
sweep_configuration = {
    'method' : 'bayes',
    'metric' : { 'goal' : 'maximize',
    'name' : 'Val_Accuracy'},
    'parameters':{
        'batchsize' : {'values' : [64,128,256]},
        'input_embedding_size' : {'values' : [128,256,512]},
        'no_of_layers' : {'values' : [2,3,4,5]},
        'hidden_size' : {'values' : [128,256,512]},
        'cell_type' : {'values' : ['lstm','rnn','gru']},
        'bidirectional' : {'values' : ['Yes']},
        'dropout' : {'values' : [0.2,0.4]},
        'epochs' : {'values' : [10,15,20]}
    }
}
sweep_id = wandb.sweep(sweep = sweep_configuration,project = 'DLAssignment3')
wandb.agent(sweep_id,function=sweep)

# Encoder Decoder Seq2seq for Attention

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size,hidden_size,embedding_size,num_layers,typecell,dropout,bidirectional):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout)
        self.num_layers=num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.bidirectional=bidirectional
        self.p=dropout
        self.typecell=typecell
        if self.typecell=="gru":
            self.step = nn.GRU(embedding_size, hidden_size,num_layers,dropout=self.p,bidirectional=self.bidirectional)    
        if self.typecell=="lstm":
            self.step = nn.LSTM(embedding_size, hidden_size,num_layers,dropout=self.p,bidirectional=self.bidirectional)
        if self.typecell=="rnn":
            self.step = nn.RNN(embedding_size, hidden_size,num_layers,dropout=self.p,bidirectional=self.bidirectional) 

    def forward(self, inp, hidden,cell=None):
        embedded = self.dropout(self.embedding(inp))
        if self.typecell=="gru":
            output, hidden = self.step(embedded, hidden)   
            return output,hidden
        if self.typecell=="rnn":
            output, hidden = self.step(embedded, hidden)   
            return output,hidden
        if self.typecell=="lstm": 
            output, (hidden,cell) = self.step(embedded, (hidden,cell))
            return output, (hidden,cell)

    def initHidden(self):
        #for bidirection
        num_layers=self.num_layers
        if self.bidirectional:
            num_layers=self.num_layers*2
        hidden=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
        if self.typecell=="lstm":
            cell=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
            return (hidden,cell)        
        return hidden


        
class Attention(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_layers,output_size,typecell,dropout):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(0.2)
        self.num_layers=num_layers
        self.output_size=output_size
        self.typecell=typecell
        self.p=dropout
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.u=nn.Linear(hidden_size,hidden_size)
        self.w=nn.Linear(hidden_size,hidden_size)
        self.v=nn.Linear(hidden_size,1)
        if self.typecell=="gru":
            self.step = nn.GRU(hidden_size+embedding_size, hidden_size,num_layers,dropout=self.p)   
        if self.typecell=="rnn":
            self.step = nn.RNN(hidden_size+embedding_size, hidden_size,num_layers,dropout=self.p)   
        if self.typecell=="lstm":
            self.step = nn.LSTM(hidden_size+embedding_size, hidden_size,num_layers,dropout=self.p) 
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, inp, hidden,encoder_output,cell=None):
        embedded = self.embedding(inp)
        u1=self.u(encoder_output)
        w1=self.w(hidden[-1])
        z=nn.Tanh()(u1+w1.resize(1,BATCH_SIZE,self.hidden_size))
        et=self.v(z)
        alpha=nn.Softmax(dim=0)(et)
        ct=torch.einsum("snk,snl->knl", alpha, encoder_output)
        ctet=torch.cat((ct,embedded),dim=2)
        if self.typecell=="gru":
            output, hidden = self.step(ctet, hidden)  
            output1=self.out(output)
            return output1,hidden,alpha
        if self.typecell=="rnn":
            output, hidden = self.step(ctet, hidden)  
            output1=self.out(output)
            return output1,hidden,alpha
        if self.typecell=="lstm": 
            output, (hidden,cell) = self.step(ctet,  (hidden,cell))
            output1=self.out(output)
            return output1, (hidden,cell),alpha

    
    def initHidden(self):
        num_layers=self.num_layers
        hidden=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
        if self.typecell=="lstm":
            cell=torch.zeros(num_layers,BATCH_SIZE,self.hidden_size, device=device)
            return (hidden,cell)        
        return hidden

    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder,hencoder,cell=None):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.hencoder=hencoder
        self.cell=cell
    def forward(self, inp, target,teacher_force_ratio,capture=True):
        outputs = torch.zeros(HINDILEN,BATCH_SIZE ,len(hindivocab)).to(device)
        if self.encoder.typecell=="lstm":
            p,(hencoder3d,cell)=self.encoder.forward(inp.to(device),self.hencoder,self.cell)
        else:
            p,hencoder3d=self.encoder.forward(inp.to(device),self.hencoder)
        p=torch.split(p,[self.encoder.hidden_size,self.encoder.hidden_size],dim=2)
        p=torch.add(p[0],p[1])/2
        tempdecoder=torch.zeros(self.encoder.num_layers,BATCH_SIZE,hencoder3d.size()[2]).to(device)
        tempdecoder[0]=hencoder3d[hencoder3d.size()[0]//2-1]
        tempdecoder[1]=hencoder3d[(hencoder3d.size()[0]//2)*2-1]
        hdecoder=torch.add(tempdecoder[0],tempdecoder[1])/2
        hdecoder=hdecoder.repeat(self.decoder.num_layers,1,1)
        if self.encoder.typecell=="lstm":
            tempcell=torch.zeros(2,BATCH_SIZE,cell.size()[2]).to(device)
            tempcell[0]=hencoder3d[cell.size()[0]//2-1]
            tempcell[1]=hencoder3d[(cell.size()[0]//2)*2-1]
            cell=torch.add(tempcell[0],tempcell[1])/2
            cell=cell.repeat(self.decoder.num_layers,1,1)
        x=torch.full((1,BATCH_SIZE),hindidictc['0'])
        g=[]
        if self.encoder.typecell=="lstm":
            output,(hdecoder,cell),alpha=self.decoder.forward(x.to(device),hdecoder,p.to(device),cell)
            if capture:
                g.append(alpha)
        else:
            output,hdecoder,alpha=self.decoder.forward(x.to(device),hdecoder,p.to(device))
            g.append(alpha)
        outputs[0]=output
        t=1
        for i in range(1,HINDILEN):
            if random.random() > teacher_force_ratio:
                output=self.decoder.softmax(output)
                nextinp=torch.argmax(output, dim=2)
                if self.encoder.typecell=="lstm":
                    output,(hdecoder,cell),alpha=self.decoder.forward(nextinp.to(device),hdecoder,p.to(device),cell)
                    if capture:
                        g.append(alpha)                    
                    
                else:
                    output,hdecoder,alpha=self.decoder.forward(nextinp.to(device),hdecoder,p.to(device))
                    g.append(alpha)
                outputs[t]=output
                t+=1
            else:
                nextinp=target[i-1,:].unsqueeze(0)
                if self.encoder.typecell=="lstm":
                    output,(hdecoder,cell),alpha=self.decoder.forward(nextinp.to(device),hdecoder,p.to(device),cell)
                    if capture:
                        g.append(alpha)  
                else:
                    output,hdecoder,alpha=self.decoder.forward(nextinp.to(device),hdecoder,p.to(device))
                    g.append(alpha)
                outputs[t]=output
                t+=1
        return outputs,g
        
    
alpha=[]
def train(encoder,decoder,seq2seq,epoch):
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(reduction="sum")
    loss=0
    count=0
    numbatches=englishwords.shape[0]//BATCH_SIZE
    for ep in range(epoch):
        trainloss=0
        train_correct=0
        for i in range(numbatches):
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            temp=englishwords[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
            temph=hindiwords[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
            temp=temp.t()
            temph=temph.t()
            global alpha
            output,alpha=seq2seq.forward(temp,temph,0.5)
            train_correct+=train_accuracy(output,temph)
            output = output[:].reshape(-1, output.shape[2])
            tem = temph[:].reshape(-1)
            loss=criterion(output,tem)
            loss.backward()
            trainloss+=loss.item()
            torch.nn.utils.clip_grad_norm_(decoder.parameters(),max_norm = 1)
            torch.nn.utils.clip_grad_norm_(encoder.parameters(),max_norm = 1)
            encoder_optimizer.step()
            decoder_optimizer.step()
        val_correct,cur_loss=accuracy(seq2seq,englishwordsval,hindiwordsval)
        print(ep,trainloss/(51200*HINDILEN),cur_loss/(4096*HINDILEN),val_correct,train_correct)
        trainloss=trainloss/(51200*HINDILEN)
        cur_loss=cur_loss/(4096*HINDILEN)
        tra_acc=train_correct/51200
        val_acc=val_correct/4096
        wandb.log({"Training loss":trainloss,'Val loss':cur_loss,'Training Accuracy':tra_acc,'Val Accuracy':val_acc})

def train_accuracy(output,temph):
        output=nn.Softmax(dim=2)(output)
        output=torch.argmax(output,dim=2)
        temph=temph.t()
        output=output.t()
        correct=0
        for i in range(BATCH_SIZE):
            if(torch.equal(output[i],temph[i])):
                correct+=1
        return correct
        
def accuracy(seq2seq,english,hindi):
    numbatches=english.shape[0]//BATCH_SIZE
    correct=0
    criterion = nn.CrossEntropyLoss(reduction="sum")
    loss=0
    for i in range(numbatches):
        temp=english[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        temph=hindi[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        temp=temp.t()
        temph=temph.t()
        global alpha
        output,alpha=seq2seq.forward(temp,temph,0)
        o = output[:].reshape(-1, output.shape[2])
        tem = temph[:].reshape(-1)
        x=criterion(o,tem)
        loss+=x.item()
        output=nn.Softmax(dim=2)(output)
        output=torch.argmax(output,dim=2)
        temph=temph.t()
        output=output.t()
        for i in range(BATCH_SIZE):
            if(torch.equal(output[i],temph[i])):
                correct+=1

    return correct,loss


# Saving File for predictions

In [ ]:
accuracy(seq2seq,englishwordstest,hindiwordstest)

In [ ]:

def filetest(seq2seq,english,hindi):
    numbatches=english.shape[0]//BATCH_SIZE
    correct=0
    criterion = nn.CrossEntropyLoss(reduction="sum")
    loss=0
    for i in range(numbatches):
        temp=english[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        temph=hindi[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        temp=temp.t()
        temph=temph.t()
        output,alpha=seq2seq.forward(temp,temph,0)
        o = output[:].reshape(-1, output.shape[2])
        tem = temph[:].reshape(-1)
        x=criterion(o,tem)
        loss+=x.item()
        output=nn.Softmax(dim=2)(output)
        output=torch.argmax(output,dim=2)
        temph=temph.t()
        output=output.t()
        temp=temp.t()
        for i in range(BATCH_SIZE):
            eng=[]
            for y in temp[i]:
                if y==2:
                    break
                eng.append(englishdicti[y.item()])
            hindip=[]
            for y in output[i]:
                if y==1:
                    break
                hindip.append(hindidicti[y.item()])       
            hinditrue=[]
            for y in temph[i]:
                if y==1:
                    break
                hinditrue.append(hindidicti[y.item()]) 
            hindip=''.join(hindip)
            hinditrue=''.join(hinditrue)
            eng=''.join(eng)
            with open('output4.txt', 'a', encoding='utf-8') as file:
                file.write(eng+' '+hinditrue+' '+hindip+'\n')
            if(torch.equal(output[i],temph[i])):
                correct+=1

    return correct,loss

In [ ]:
filetest(seq2seq,englishwordstest,hindiwordstest)

# Attention Heatmap

In [ ]:
accuracy(seq2seq,englishwordstest[:128],hindiwordstest[:128])

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
fig,axes=plt.subplots(3, 3, figsize=(12, 12))
figsize=(64,64)
fig.tight_layout(pad=5.0)
fig.subplots_adjust(top=0.90)

axes = axes.ravel()
def plot_rowwise_heatmap(matrix,eng,hindi,i):
    rows, cols = matrix.shape

    # Create a figure and axis

    # Set the colormap
    cmap = sns.color_palette("viridis")

    # Plot the heatmap
    sns.heatmap(matrix, cmap=cmap,  ax=axes[i], cbar=True, cbar_kws={'label': 'Value'})
    hindi_font = FontProperties(fname = os.getcwd()+'/nirmala.ttf')

    axes[i].set_xticks(np.arange(cols) + 0.5)
    axes[i].set_xticklabels(hindi,fontproperties=hindi_font)


    axes[i].set_yticks(np.arange(rows) + 0.5)
    axes[i].set_yticklabels(eng)

    # Add a colorbar
    cbar = axes[i].collections[0].colorbar
    cbar.set_label('Value')



alpha1=torch.cat(tuple(x for x in alpha),dim = 2).to(device)
alpha2=alpha1.permute(1,0,2)
# print(alpha2[0].size())
x=alpha2[0].cpu().detach().numpy()
plot=[]
for i in range(9):
    eng=[]
    for y in englishwordstest[i]:
        if y==2:
            break
        eng.append(englishdicti[y.item()])
    hindi=[]
    for y in hindiwordstest[i]:
        if y==1:
            break
        hindi.append(hindidicti[y.item()])
    plot_rowwise_heatmap(x[0:len(eng),0:len(hindi)],eng,hindi,i)
    

fig.suptitle('Attention Mechanism(HeatMap)',fontsize=20)
plt.savefig('plot.png')
plt.show()

# Sweep code for Attention

In [ ]:
def sweep():
    wandb.init(project='DLAssignment3')
    config = wandb.config
    wandb.run.name = "Attn_cellType_{}_embSize_{}_layers_{}_batchSize_{}_hidden_{}_dropout_{}".format(config.cell_type,config.input_embedding_size,config.no_of_layers,config.batchsize,config.hidden_size,config.dropout)
    hidden_size = config.hidden_size
    char_embed_size = config.input_embedding_size
    no_of_layers = config.no_of_layers
    epochs = config.epochs
    batchsize = config.batchsize
    dropout = config.dropout
    cell_type=config.cell_type
    if(config.bidirectional=='Yes'):
        bidirectional=True
    else:
        bidirectional=False
    encoder=EncoderRNN(len(englishvocab),hidden_size,char_embed_size,no_of_layers,cell_type,dropout,bidirectional).to(device)
    decoder=Attention(len(hindivocab),hidden_size,char_embed_size,no_of_layers,len(hindivocab),cell_type,dropout).to(device)
    if cell_type=='lstm':
        hencoder,cell=encoder.initHidden()
        seq2seq=Seq2Seq(encoder,decoder,hencoder,cell)
    else:
        hencoder=encoder.initHidden()
        seq2seq=Seq2Seq(encoder,decoder,hencoder)
    train(encoder,decoder,seq2seq,epochs)


In [ ]:
sweep_configuration = {
    'method' : 'bayes',
    'metric' : { 'goal' : 'maximize',
    'name' : 'Val Accuracy'},
    'parameters':{
        'batchsize' : {'values' : [128]},
        'input_embedding_size' : {'values' : [256,128,512]},
        'no_of_layers' : {'values' : [2,3,4,5]},
        'hidden_size' : {'values' : [256,128,512]},
        'cell_type' : {'values' : ['lstm','gru','rnn']},
        'bidirectional' : {'values' : ['Yes']},
        'dropout' : {'values' : [0.2,0.4]},
        'epochs' : {'values' : [15,20]}
    }
}
sweep_id = wandb.sweep(sweep = sweep_configuration,project = 'DLAssignment3')
wandb.agent(sweep_id,function=sweep,count)
